In [1]:
import pandas as pd
from datetime import timedelta, datetime
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import prepare as p

import warnings
warnings.filterwarnings("ignore")

import acquire as a

In [2]:
if os.path.isfile('tsa_store_data.csv'):
    print('File exists, pulling from system.')
    store = pd.read_csv('tsa_store_data.csv')
else:
    print('File does not exist, extracting from MySQL.')
    store = a.get_store_data()

File exists, pulling from system.


In [3]:
store = p.prepare_store(store)

In [4]:
store.head()

,item_upc14,item_upc12,item_brand,item_name,item_price,sale_id,sale_amount,store_address,store_zipcode,store_city,store_state,month,year,sales_total
sale_date,,,,,,,,,,,,,,
2013-01-01,35200264013,35200264013,Riceland,Riceland American Jazmine Rice,0.84,1,13,12125 Alamo Ranch Pkwy,78253,San Antonio,TX,1,2013,13.84
2013-01-02,35200264013,35200264013,Riceland,Riceland American Jazmine Rice,0.84,2,11,12125 Alamo Ranch Pkwy,78253,San Antonio,TX,1,2013,11.84
2013-01-03,35200264013,35200264013,Riceland,Riceland American Jazmine Rice,0.84,3,14,12125 Alamo Ranch Pkwy,78253,San Antonio,TX,1,2013,14.84
2013-01-04,35200264013,35200264013,Riceland,Riceland American Jazmine Rice,0.84,4,13,12125 Alamo Ranch Pkwy,78253,San Antonio,TX,1,2013,13.84
2013-01-05,35200264013,35200264013,Riceland,Riceland American Jazmine Rice,0.84,5,10,12125 Alamo Ranch Pkwy,78253,San Antonio,TX,1,2013,10.84
